# HelioCloud at AGU 2024
## Big data, burst capability, collaboration, science-in-the-browser

This notebook showcases HelioCloud, the NASA-created cloud platform for big data, high end computing, and data collaboration that any institution can set up.  It features:

1) Science in your browser with a Python environment that 'just works'
2) Ability to search and find data from many missions
3) Computing on datasets without having to transfer them to your laptop
4) Analysis and plotting interactively in Jupyter Notebooks
5) Spinning up 100+ temporary CPUs for a big analysis task using Dask
6) Examples from many of the PyHC core packages, such as HAPI, SunPy, Kamodo, PlasmaPy, pySat, PySpedas, and SpacePy

In [ ]:
# ensure we have the proper versions, and plotting capability
import os
import pkg_resources
try:
    pkg_resources.require("cloudcatalog>0.5.0")  # modified to use specific numpy
except:
    %pip install cloudcatalog --upgrade
    os._exit(00)
import matplotlib.pyplot as plt
# optional, prettier inline Notebook plotting
%matplotlib inline
%config InlineBackend.figure_formats = ['svg'] 
import warnings
warnings.filterwarnings( "ignore")

# HelioCloud shared cloud file registry (cloudcatalog)

This is a simple standard for any dataset that enables users to access it via an API or directly.  The short definition is:

    * Anyone can publish an S3 disk with a 'catalog.json' describing their datasets
    * Each dataset has a flat-file <dataid>_YYYY.csv index of its contents
    * These fetchable indexes have the form "start, stop, file_location, filesize" (plus optional metadata)
    * A python client allows easy search and fetch using data IDs and time ranges
    
Here is an example querying for the filelists for MMS and for AIA for all of Aug 2020.

In [ ]:
import cloudcatalog
mms_ids = cloudcatalog.EntireCatalogSearch().search_by_id('srvy_ion')
aia_ids = cloudcatalog.EntireCatalogSearch().search_by_keywords(['AIA 0193'])
print(f"ID search: found {len(mms_ids)} IDs matching 'srvy_ion' and {len(aia_ids)} IDs matching 'AIA 0193', picking first of each set.")

mmsID, aiaID = mms_ids[0]['id'], aia_ids[0]['id']
start, stop = '2020-08-01T00:00:00Z', '2020-08-30T00:00:00Z'
fr=cloudcatalog.CloudCatalog("s3://gov-nasa-hdrl-data1/")
mms_files = fr.request_cloud_catalog(mmsID, start_date=start, stop_date=stop)
aia_files = fr.request_cloud_catalog(aiaID, start_date=start, stop_date=stop)
print(f"Filelists: found {len(mms_files)} {mmsID} files, {len(aia_files)} {aiaID} files for time range {start} to {stop}")
file1 = mms_files.iloc[0]['datakey']
print(f"\nFor example, here is our {mmsID} files")
mms_files

## Let's do something useful, like plot an MMS FEEPS electron survey files and a pair of AIA 0193Angstrom EUV images

Here we go.  For MMS can use the 'cdflib' to remotely read the cloud-stored MMS datafile and plot it as usual.  This is identical to working with CDFs on your laptop, but with no network transfers. The data and the computation occur in the AWS cloud. We plot several FEEPS variables.

In [ ]:
import cdflib
from matplotlib import pyplot as plt
with cdflib.CDF(file1) as fin:
    varnames = fin.cdf_info()['zVariables']
    x = fin.varget("epoch")
    y = fin.varget(varnames[5])
    plt.xlabel("Index")
    plt.ylabel(varnames[5])
    plt.title(f"Plot of {varnames[5]}")
    plt.plot(x,y)
    plt.show()


Now images from AIA's EUVI.  Here we generate then plot AIA EUV images using AstroPy to read and PyPlot to plot.  Again, we are not copying the disk to our machine, but reading directly from the cloud to plot.

In [ ]:
import astropy.io.fits
import matplotlib
from matplotlib import pyplot as plt
import sunpy.visualization.colormaps as cm
import numpy as np

cmap = matplotlib.colormaps['sdoaia193']
for i in range(0,2200,700): #ele in aia_files.iloc[1:4]: #np.nditer(aia_files, REFS_OK=True):
    fname = aia_files.iloc[i]['datakey']
    try:
        hdul = astropy.io.fits.open(fname)
    except:
        import s3fs
        fs=s3fs.S3FileSystem(anon=True)
        hdul = astropy.io.fits.open(fs.open(fname))
    if i > 0:
        plt.imshow(np.log(hdul[1].data), cmap=cmap)
        plt.colorbar()
        plt.show()

## We can also use PyHC packages in this browser.

We've already shown AstroPy, and now we give examples for HAPI time series data, SunPy image data, and Kamodo model generation.  These HAPI time series fetches or SunPy image calls or Kamodo model generations, executed within this Notebook, are the same as it would be on your laptop.  

For HAPI, we look at DST1800 and Proton_QI1800 from OMNIWeb, from the 2022 PyHC Summer School tutorial.

In [ ]:
from hapiclient import hapi
from hapiplot import hapiplot
###    import math
# HAPI test, OMNIWeb data
# The data server
server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
# The data set
dataset    = 'OMNI2_H0_MRG1HR'
# Start and stop times
start      = '2021-10-25T00:00:00Z'
stop       = '2021-12-01T00:00:00Z'
# The HAPI convention is that parameters is a comma-separated list. Here we request two parameters.
parameters = 'DST1800,Proton_QI1800'
# Configuration options for the hapi function.
opts = {'logging': False, 'usecache': True, 'cachedir': './hapicache' }
# Get parameter data. See section 5 for for information on getting available datasets and parameters
data, meta = hapi(server, dataset, parameters, start, stop, **opts)
meta = hapiplot(data, meta)

### SunPy
This is the SunPy AIA Demo from https://docs.sunpy.org/en/stable/tutorial/maps.html, fetching a single demo AIA 171A EUV image and adding in WCS and prettier plotting.  Again the code to run this is identical to your laptop code, because HelioCloud is about science.

In [ ]:
import sunpy.map
import sunpy.data.sample
import astropy.units as u
import matplotlib.pyplot as plt
sunpy.data.sample.AIA_171_IMAGE
my_map = sunpy.map.Map(sunpy.data.sample.AIA_171_IMAGE)
fig = plt.figure()
ax = fig.add_subplot(projection=my_map)
my_map.plot(axes=ax, clip_interval=(1, 99.5)*u.percent)
plt.colorbar()
plt.show()

### Kamodo
Here we run the Kamodo model generator example from the 2022 PyHC Summer School.  Note the use of interactive plots-- you can use the mouse to rotate or change the plot size.

In [ ]:
from kamodo import Kamodo
import numpy as np
x, y, z = np.meshgrid(np.linspace(-2,2,4),
                      np.linspace(-3,3,6),
                      np.linspace(-5,5,10))
points = np.array(list(zip(x.ravel(), y.ravel(), z.ravel())))
def fvec_Ncomma3(rvec_Ncomma3 = points):
    return rvec_Ncomma3
k = Kamodo(fvec_Ncomma3 = fvec_Ncomma3)
k.plot('fvec_Ncomma3')

## Ability to run a task with Lambdas (lambdas test) 

Python Lambdas are a quick way to run a function on a large set of data.  Here we do a very simple checksum on MMS files fetched from a CloudCatalog query.  You could just as easily add a function that does a more meaningful calculation, then run it on all the data with one line of invoking code.

Again, with HelioCloud we are skipping the usual sluggish copying over of data before processing, and instead doing the computation on the data directly in the cloud.  This provides you with ready access to data, avoids filling disks, is faster, and lets you share code so others can collaborate (they'll already have access to the Petabytes of data in HelioCloud's public cloud archive).

In [ ]:
import cloudcatalog
fr=cloudcatalog.CloudCatalog("s3://gov-nasa-hdrl-data1/")
dataset_id1 = 'mms1_feeps_brst_electron'
start = '2020-08-01T00:00:00Z'
stop =   '2020-08-30T00:00:01Z'
file_registry1 = fr.request_cloud_catalog(dataset_id1, start_date=start, stop_date=stop, overwrite=False)
print(f"{len(file_registry1)} files, operating lambda on first ten.")
print('Python Hash of File | Start Date | File Size')

fr.stream(file_registry1[0:10], lambda bo, d, e, f: print(hash(bo.read()), d.replace(' ', 'T')+'Z', e, f))

In [ ]:
if 'imports_loaded_flag' not in locals():
    import pysat
# Testing out the xarray installation
inst = pysat.Instrument('pysat', 'testmodel')
inst.load(2009, 1)
assert sum(sum(sum(inst.data['dummy1'].values))) >= 945378

### PySpedas
Requires "import pyspedas" and "from pytplot import tplot, get_data", also "import math" for the assertion math.isclose() function.

In [ ]:
if 'imports_loaded_flag' not in locals():
    import pyspedas
    from pytplot import tplot, get_data
    import math
time_range = ['2020-04-20/06:00', '2020-04-20/08:00']
pyspedas.solo.mag(trange=time_range, time_clip=True)
pyspedas.mms.fgm(trange=time_range, time_clip=True, probe=2)
tplot(['B_RTN','mms2_fgm_b_gsm_srvy_l2_bvec'])
mag_data = get_data('mms2_fgm_b_gse_srvy_l2_bvec')
assert math.isclose(sum(sum(mag_data.y)),152229.33203125)

# Cloud 'throw lots of CPUs at a problem' (Dask cluster) 'burst' test

A big advantage of HelioCloud is the ability to send a computation to multiple CPUs, then gather the results.  This uses a library called Dask, which looks similar to a Python Lambda.  For example, given a function that does science 'process_fits_s3' and  a list of files 's3_files', the code is simply:
```
time_irrad = client.map(process_fits_s3, s3_files)
all_data = client.gather(time_irrad)
```
There is an initial time hit to spin up the Cluster of extra CPUs, but then this and subsequent jobs are very rapid. Our code block 1 sets up the data environment and has our 'DO_SCIENCE' analysis routines. Block 2 starts the cluster.  Block 3 does the dask run (and can be re-run).  Once done, Block 4 shuts down the cluster for good.

In [ ]:
import cloudcatalog
import boto3
import dask
import io
import time
import re
import astropy.io.fits
from dask.distributed import Client
from dask_gateway import Gateway, GatewayCluster
    
def DO_SCIENCE(mydata):
    # Calculates total irradiance from an image.  You can put better science here
    iirad = mydata.mean()
    return iirad

def process_fits_s3(s3key:str): # -> Tuple[str, float]:
    """ grabs an S3 file then runs DO_SCIENCE() on it """
    sess = boto3.session.Session() # do this each open to avoid thread problem 'credential_provider'
    s3c = sess.client("s3")
    [mybucket,mykey] = re.sub(r"s3://","",s3key).split("/",1)
    try:
        fobj = s3c.get_object(Bucket=mybucket,Key=mykey)
        rawdata = fobj['Body'].read()
        bdata = io.BytesIO(rawdata)
        hdul = astropy.io.fits.open(bdata,memmap=False)        
        date = hdul[1].header['T_OBS']
        irrad = DO_SCIENCE(hdul[1].data)
    except:
        print("Error fetching ",s3key)
        date, irrad = None, None       
    return date, irrad

frID = "aia_0094"
fr=cloudcatalog.CloudCatalog("s3://gov-nasa-hdrl-data1/")
file_registry1 = fr.request_cloud_catalog(frID, start_date=start, stop_date=stop, overwrite=False)
filelist = file_registry1['datakey'].to_list()
s3_files = filelist[0:100] # small test set to test

In [ ]:
""" Now we initialize the Dask gateway and cluster, using your above parameters, to set up the virtual machines that will subsequently operate on the data. We use some non-optimized Dask parameters for setting up the run."""
gateway = Gateway()
options = gateway.cluster_options()
options.worker_cores, options.worker_memory = 2, 1
cluster = gateway.new_cluster(options)
client=Client(cluster)
cluster.adapt(minimum=1, maximum=4)
client

In [ ]:
now=time.time()
try:
    time_irrad = client.map(process_fits_s3, s3_files) # do it
    all_data = client.gather(time_irrad) # gather results
    print("Done! Completed in time ",(time.time()-now)/60.0,"minutes, on",len(all_data),"files")
except:
    print("Cluster needs to be started (or re-started) before running code.")

In [ ]:
yn=input("Do you want to shut down the cluster, or re-use it? (y/r)?")
if yn == 'y':
    cluster.shutdown()

# Recap

To recap, HelioCloud lets you do the same science as on your laptop using the same Python packages you already use, but in the cloud. This means a stable pre-loaded Python environment, fast access to Terabyte- and Petabyte-sized datasets, and the ability to throw lots of CPUs at data- or computation-heavy problems. Because everyone using a HelioCloud has the same software environment and data access rights, you can collaborate more easily because you only need to share code, not envs and datasets.

The HelioCloud 'stack' runs on AWS and is released as open source, so any institution can install it.  It is available via heliocloud.org.  HelioCloud is a NASA-funded software development effort and eagerly seeks community input.